In [ ]:
import pandas as pd

DAY = 24*60*60

In [145]:
df = pd.read_csv('./connection_graph.csv', index_col=0, low_memory=False)
df

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383
...,...,...,...,...,...,...,...,...,...
996209,967,18:38:00,18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543
996210,967,18:39:00,18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808
996211,967,18:41:00,18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584
996212,967,18:42:00,18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523


In [ ]:
# rm the first column and duplicate rows
# df = df.drop(df.columns[0], axis=1)
# df.drop_duplicates(inplace=True)
df
# save to file

In [ ]:
# df.to_csv('./connection_graph.csv')

In [146]:
def get_sec_from_str(time_str):
    time_str = time_str.split(':')
    return (int(time_str[0]) * 3600 + int(time_str[1]) * 60 + int(time_str[2])) % DAY

df['departure_time_sec'] = df['departure_time'].apply(get_sec_from_str)
df['arrival_time_sec'] = df['arrival_time'].apply(get_sec_from_str)
df

,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon,departure_time_sec,arrival_time_sec
0,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539,75120,75180
1,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735,75180,75240
2,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376,75240,75300
3,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617,75300,75420
4,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383,75420,75540
...,...,...,...,...,...,...,...,...,...,...,...
996209,967,18:38:00,18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543,67080,67140
996210,967,18:39:00,18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808,67140,67260
996211,967,18:41:00,18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584,67260,67320
996212,967,18:42:00,18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523,67320,67380


In [ ]:
stops = df['start_stop'].unique()
stops, stops.shape

In [ ]:
import numpy as np

import networkx as nx

graph = nx.DiGraph()
for stop in stops:
    graph.add_node(stop)

for number, edge in df.iterrows():
  if not graph.has_edge(edge['start_stop'], edge['end_stop']):
    graph.add_edge(edge['start_stop'], edge['end_stop'], schedule=[])

  graph[edge['start_stop']] [edge['end_stop']] ['schedule'].append({
    'dep': edge['departure_time_sec'],
    'line': edge['line'],
    'end_stop_lat': edge['end_stop_lat'],
    'end_stop_lon': edge['end_stop_lon'],
    'arrival': edge['arrival_time_sec'],
    'id': number
    })


def convert_to_dataframe(graph):
  for start, stop in graph.edges:
    graph[start][stop]['schedule'] = pd.DataFrame(graph[start][stop]['schedule'])

convert_to_dataframe(graph)

In [98]:
def print_graph(graph):
  for start, stop in graph.edges:
    print(start, '->', stop)
    print(graph[start][stop]['schedule'])
# print_graph(graph)

Zajezdnia Obornicka -> Paprotna
       dep line  end_stop_lat  end_stop_lon  arrival      id
0    75120    A     51.147752     17.020539    75180       0
1    78360    A     51.147752     17.020539    78420      81
2    73320    A     51.147752     17.020539    73380     125
3    20160    A     51.147752     17.020539    20220     202
4    19560    A     51.147752     17.020539    19620     689
..     ...  ...           ...           ...      ...     ...
293  76440  602     51.147752     17.020539    76500  934107
294  16020  714     51.147752     17.020539    16080  939524
295  16620  714     51.147752     17.020539    16680  939975
296  17220  714     51.147752     17.020539    17280  940426
297  22620  714     51.147752     17.020539    22680  940862

[298 rows x 6 columns]
Zajezdnia Obornicka -> POŚWIĘTNE
       dep line  end_stop_lat  end_stop_lon  arrival      id
0    79500    D     51.150667     17.028956    79620   13998
1    77700    D     51.150667     17.028956    77820   14

In [166]:
class MinProperties:
  def reset(self):
    self.min = float('inf')
    self.min_diff = float('inf')
    self.min_idx = 0

min_properties = MinProperties()  
def difference(row, departure):
    global min_properties
    arrival = row['arrival']
    new_departure = row['dep']

    # jeśli przyjechałeś o 23:55, to nie wyjedziesz o 23:50 (1)
    # jeśli przyjechałeś o 00:05, to nie wyjedziesz o 23:55 (2)
    if departure > new_departure or new_departure - departure > 20 * 60 * 60:
        return row

    value = (arrival - departure + DAY) % DAY
    
    if value < min_properties.min_diff:
        min_properties.min = arrival
        min_properties.min_diff = value
        min_properties.min_idx = row['id']
    return row


In [167]:
from typing import Dict

min_properties = MinProperties()  

class NodeFeatures:
  def __init__(self, id, name: str, time: float, prev):
    self.id = id
    self.name = name
    self.time = time # departue time
    self.prev = prev

  def set_time(self, departure, edge: float, prev):
    time, id = self.get_best_time(edge, departure)

    if time < self.time:
      self.prev = prev
      self.id = id
      self.time = time

  def get_best_time(self, edge, departure):
    schedule = edge['schedule']

    min_properties.reset()
    schedule.apply(difference, axis=1, args=(departure,))

    return min_properties.min, min_properties.min_idx

def get_best_path(visited: Dict[str, NodeFeatures], destination: str):
  start = visited[destination]
  path = [start]

  while start.prev:
    start = start.prev
    path.append(start)

  return path[::-1]


np.seterr(divide='ignore', invalid='ignore')

def djikstra(graph: nx.Graph, start: str, destination: str, starting_time: int):
  unvisited: Dict[str, NodeFeatures] = {node: NodeFeatures(-1, node, np.inf, None) for node in graph.nodes}
  visited: Dict[str, NodeFeatures] = {}

  start_stop = unvisited.pop(start)
  start_stop.time = starting_time

  while unvisited:
    neighbours = graph[start_stop.name]
    best_neighbour = NodeFeatures(-1, "", np.inf, None)
    for neigbour in neighbours:
      if not neigbour in unvisited:
        continue

      unvisited[neigbour].set_time(start_stop.time, neighbours[neigbour], start_stop)

      # ex 23:50 > 00:10 ... and best_neighbour.time - unvisited[neigbour].time < 10 * 60 * 60
      if best_neighbour.time > unvisited[neigbour].time:
        best_neighbour = unvisited[neigbour]

    if best_neighbour.time == np.inf:
      start_stop = unvisited.popitem()[1]
      continue

    visited[start_stop.name] = start_stop
    start_stop = unvisited.pop(best_neighbour.name)

  return get_best_path(visited, destination)


visited = djikstra(graph, 'FAT', 'GALERIA DOMINIKAŃSKA', 10 * 60 * 60 + 5 * 60)
visited

In [169]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

node = None
for idx, node in enumerate(visited):
  # print(node.name, node.time)
  try:
    row = df.loc[node.id]
    # display only the most important columns: line, start_stop, end_stop, departure_time, arrival_time
    print(row[['line', 'start_stop', 'end_stop', 'departure_time', 'arrival_time']].values)

  except:...
    # print(row[['line', 'start_stop', 'end_stop', 'departure_time', 'arrival_time']].columns.values)
  print()


['5' 'FAT' 'Hutmen' '10:05:00' '10:06:00']

['5' 'Hutmen' 'Bzowa (Centrum Historii Zajezdnia)' '10:06:00' '10:07:00']

['5' 'Bzowa (Centrum Historii Zajezdnia)' 'pl. Srebrny' '10:07:00'
 '10:08:00']

['5' 'pl. Srebrny' 'Stalowa' '10:08:00' '10:09:00']

['5' 'Stalowa' 'Pereca' '10:09:00' '10:11:00']

['5' 'Pereca' 'Grabiszyńska' '10:11:00' '10:12:00']

['5' 'Grabiszyńska' 'Kolejowa' '10:12:00' '10:13:00']

['5' 'Kolejowa' 'pl. Legionów' '10:13:00' '10:15:00']

['5' 'pl. Legionów' 'Arkady (Capitol)' '10:15:00' '10:17:00']

['D' 'Arkady (Capitol)' 'GALERIA DOMINIKAŃSKA' '10:20:00' '10:25:00']



In [ ]:
departure = 34900

df2 = pd.DataFrame(0, index=np.arange(df.shape[0]), columns=['diff'])
df2['diff'] = (df['departure_time_sec'] + DAY - departure) % DAY

# df2['diff'].min() find min and index of min
df2[df2['diff'] > 0].idxmin().iloc[0]

In [179]:
from typing import Dict

min_properties = MinProperties()

class NodeFeatures:
  def __init__(self, id, name: str, time: float, prev):
    self.id = id
    self.name = name
    self.cost = np.inf
    self.time = time # departue time
    self.prev = prev

  def set_time(self, departure, edge, prev):
    time, id = self.get_best_time(edge, departure)

    if time < self.time:
      self.prev = prev
      self.id = id
      self.time = time
      self.cost = time + calculate_manhatan(edge['schedule']['end_stop_lat'].iloc[0], edge['schedule']['end_stop_lon'].iloc[0])
      return id


  def get_best_time(self, edge, departure):
    schedule = edge['schedule']

    min_properties.reset()
    schedule.apply(difference, axis=1, args=(departure,))

    return min_properties.min, min_properties.min_idx

def get_best_path(visited: Dict[str, NodeFeatures], destination: str):
  start = visited[destination]
  path = [start]

  while start.prev:
    start = start.prev
    path.append(start)

  return path[::-1]

def calculate_manhatan(start_lat, start_long):
  global target_lat, target_long, TRAM_SPEED

  return (abs(start_lat - target_lat) + abs(start_long - target_long)) * 111_000 / TRAM_SPEED 

np.seterr(divide='ignore', invalid='ignore')

TRAM_SPEED = 16 * 1000 / 3600
target_lat = None
target_lat = None

def a_star(graph: nx.Graph, start: str, destination: str, starting_time: int):
  global target_long, target_lat
  target_lat = df[df['start_stop'] == destination].iloc[0]['start_stop_lat']
  target_long = df[df['start_stop'] == destination].iloc[0]['start_stop_lon']

  unvisited: Dict[str, NodeFeatures] = {node: NodeFeatures(-1, node, np.inf, None) for node in graph.nodes}
  visited: Dict[str, NodeFeatures] = {}

  start_stop = unvisited.pop(start)
  start_stop.time = starting_time

  while unvisited:
    neighbours = graph[start_stop.name]
    best_neighbour = NodeFeatures(-1, "", np.inf, None)
    for neigbour in neighbours:
      if not neigbour in unvisited:
        continue

      unvisited[neigbour].set_time(start_stop.time, neighbours[neigbour], start_stop)

      if best_neighbour.cost > unvisited[neigbour].cost:
        best_neighbour = unvisited[neigbour]

    if best_neighbour.name == destination:
      visited[best_neighbour.name] = best_neighbour
      break

    if best_neighbour.time == np.inf:
      start_stop = unvisited.popitem()[1]
      continue

    visited[start_stop.name] = start_stop
    start_stop = unvisited.pop(best_neighbour.name)

  return get_best_path(visited, destination)


visited = a_star(graph, 'FAT', 'GALERIA DOMINIKAŃSKA', 23 * 60 * 60 + 50 * 60)
visited

In [180]:
for idx, node in enumerate(visited):
  # print(node.name, node.time)
  try:
    row = df.loc[node.id]
    # display only the most important columns: line, start_stop, end_stop, departure_time, arrival_time
    print(row[['line', 'start_stop', 'end_stop', 'departure_time', 'arrival_time']].values)

  except:...
    # print(row[['line', 'start_stop', 'end_stop', 'departure_time', 'arrival_time']].columns.values)
  print()


['4' 'FAT' 'Fiołkowa' '23:59:00' '24:00:00']

['4' 'Fiołkowa' 'GRABISZYŃSKA (Cmentarz)' '24:00:00' '24:01:00']

['4' 'GRABISZYŃSKA (Cmentarz)' 'GRABISZYŃSKA (Cmentarz II)' '24:01:00'
 '24:02:00']

['4' 'GRABISZYŃSKA (Cmentarz II)' 'OPORÓW' '24:02:00' '24:03:00']

['251' 'OPORÓW' 'Solskiego' '24:03:00' '24:04:00']

['251' 'Solskiego' 'Wiejska' '24:04:00' '24:05:00']

['251' 'Wiejska' 'Kadłubka' '24:05:00' '24:06:00']

['251' 'Kadłubka' 'Stanki' '24:06:00' '24:07:00']

['251' 'Stanki' 'Bukowskiego' '24:07:00' '24:07:00']

['251' 'Bukowskiego' 'RACŁAWICKA' '24:07:00' '24:08:00']

['251' 'RACŁAWICKA' 'Rymarska' '24:08:00' '24:09:00']

['251' 'Rymarska' 'Wawrzyniaka' '24:09:00' '24:11:00']

['251' 'Wawrzyniaka' 'Chłodna' '24:11:00' '24:12:00']

['251' 'Chłodna' 'Sowia' '24:12:00' '24:13:00']

['251' 'Sowia' 'KRZYKI' '24:13:00' '24:15:00']

['248' 'KRZYKI' 'Orla' '24:32:00' '24:33:00']

['248' 'Orla' 'Jastrzębia' '24:33:00' '24:34:00']

['248' 'Jastrzębia' 'Hallera' '24:34:00' '24:35:00']



In [178]:

class NodeFeatures3:
  def __init__(self, id, name: str, time: float, prev):
    self.id = id
    self.name = name
    self.line = ""
    self.time = time # departue time
    self.prev = prev

  def set_time(self, departure, edge: float, prev):
    time, id = self.get_best_time(edge, departure)

    if time < self.time:
      self.prev = prev
      self.id = id['id']
      self.line = id['line']
      self.time = time
      return id

  def get_best_time(self, edge, departure):
    schedule = edge['schedule']
    try:
      connection = schedule[calculate_cost(schedule, departure)][0]
      best_time = departure.time
    except:
      connection = schedule.iloc[0]
      best_time = departure.time + 1.0

    print(connection)
    return best_time, connection

def calculate_cost(schedule, departure):
  return schedule['line'] == departure.line

def get_best_path(visited: Dict[str, NodeFeatures3], destination: str):
  start = visited[destination]
  path = [start]

  while start.prev:
    start = start.prev
    path.append(start)

  return path[::-1]

def calculate_manhatan(start_lat, start_long):
  global target_lat, target_long, TRAM_SPEED
  return (abs(start_lat - target_lat) + abs(start_long - target_long)) * 111_000 / TRAM_SPEED

np.seterr(divide='ignore', invalid='ignore')

TRAM_SPEED = 16 * 1000 / 3600
target_lat = None
target_lat = None

def a_star_changings(graph: nx.Graph, start: str, destination: str, starting_time: int):
  global target_long, target_lat
  target_lat = df[df['start_stop'] == destination].iloc[0]['start_stop_lat']
  target_long = df[df['start_stop'] == destination].iloc[0]['start_stop_lon']

  unvisited: Dict[str, NodeFeatures3] = {node: NodeFeatures3(-1, node, np.inf, None) for node in graph.nodes}
  visited: Dict[str, NodeFeatures3] = {}

  start_stop = unvisited.pop(start)
  start_stop.time = 0

  while unvisited:
    neighbours = graph[start_stop.name]
    best_neighbour = NodeFeatures3(-1, "", np.inf, None)
    for neigbour in neighbours:
      if not neigbour in unvisited:
        continue

      unvisited[neigbour].set_time(start_stop, neighbours[neigbour], start_stop)

      if best_neighbour.time > unvisited[neigbour].time:
        best_neighbour = unvisited[neigbour]

    if best_neighbour.time == np.inf:
      start_stop = unvisited.popitem()[1]
      continue

    visited[start_stop.name] = start_stop
    start_stop = unvisited.pop(best_neighbour.name)

  return get_best_path(visited, destination)


visited = a_star_changings(graph, 'DWORZEC AUTOBUSOWY', 'FAT', 34900)
visited

dep                 83640
line                    D
end_stop_lat    51.094301
end_stop_lon    17.032229
arrival             83700
id                  14034
Name: 0, dtype: object
dep                 23700
line                    K
end_stop_lat     51.08931
end_stop_lon    17.033896
arrival             23820
id                  30568
Name: 0, dtype: object
dep                 26100
line                    K
end_stop_lat    51.099602
end_stop_lon      17.0354
arrival             26220
id                  30580
Name: 0, dtype: object
dep                 33780
line                    N
end_stop_lat    51.091822
end_stop_lon     17.03285
arrival             33960
id                  40586
Name: 0, dtype: object
dep                 84300
line                    2
end_stop_lat    51.093282
end_stop_lon    17.030233
arrival             84420
id                  65040
Name: 0, dtype: object
dep                 18240
line                    4
end_stop_lat    51.099905
end_stop_lon    17.033884
a

In [ ]:
for idx, node in enumerate(visited):
  # print(node.name, node.time)
  print(node.line, node.name, node.time)
  try:
    row = df.loc[node.id]
    # display only the most important columns: line, start_stop, end_stop, departure_time, arrival_time
    print(row[['line', 'start_stop', 'end_stop', 'departure_time', 'arrival_time']].values)

  except:...
    # print(row[['line', 'start_stop', 'end_stop', 'departure_time', 'arrival_time']].columns.values)
  print()

## TABU

In [177]:
elements = ['DWORZEC AUTOBUSOWY', 'GALERIA DOMINIKAŃSKA', 'FAT', 'DWORZEC GŁÓWNY', 'DWORZEC PKP']

all_permutations = [np.random.permutation(elements) for _ in range(20)]
np.unique(all_permutations, axis=0)

array([['DWORZEC AUTOBUSOWY', 'DWORZEC PKP', 'GALERIA DOMINIKAŃSKA',
        'FAT', 'DWORZEC GŁÓWNY'],
       ['DWORZEC AUTOBUSOWY', 'FAT', 'GALERIA DOMINIKAŃSKA',
        'DWORZEC GŁÓWNY', 'DWORZEC PKP'],
       ['DWORZEC AUTOBUSOWY', 'GALERIA DOMINIKAŃSKA', 'FAT',
        'DWORZEC GŁÓWNY', 'DWORZEC PKP'],
       ['DWORZEC GŁÓWNY', 'FAT', 'GALERIA DOMINIKAŃSKA',
        'DWORZEC AUTOBUSOWY', 'DWORZEC PKP'],
       ['DWORZEC GŁÓWNY', 'GALERIA DOMINIKAŃSKA', 'DWORZEC PKP',
        'DWORZEC AUTOBUSOWY', 'FAT'],
       ['DWORZEC GŁÓWNY', 'GALERIA DOMINIKAŃSKA', 'FAT', 'DWORZEC PKP',
        'DWORZEC AUTOBUSOWY'],
       ['DWORZEC PKP', 'DWORZEC AUTOBUSOWY', 'DWORZEC GŁÓWNY',
        'GALERIA DOMINIKAŃSKA', 'FAT'],
       ['DWORZEC PKP', 'DWORZEC AUTOBUSOWY', 'GALERIA DOMINIKAŃSKA',
        'FAT', 'DWORZEC GŁÓWNY'],
       ['DWORZEC PKP', 'FAT', 'DWORZEC AUTOBUSOWY',
        'GALERIA DOMINIKAŃSKA', 'DWORZEC GŁÓWNY'],
       ['DWORZEC PKP', 'GALERIA DOMINIKAŃSKA', 'FAT', 'DWORZEC GŁÓWNY',
 

In [175]:
for permutation in np.unique(all_permutations, axis=0):
  start_time = 34900
  end_time = start_time
  for i, stop in enumerate(permutation):
    destination = permutation[(i + 1) % 3]
    visited = a_star(graph, stop, destination, end_time)
    end_time = df.loc[visited[-1].id]['arrival_time_sec']

    print('For: ', stop, 'to:', destination, 'time:', (end_time - start_time) / 60, 'minutes')

  print('For: ', permutation, 'time:', (end_time - start_time) / 60, 'minutes')

For:  DWORZEC AUTOBUSOWY to: FAT time: 25.333333333333332 minutes
For:  FAT to: GALERIA DOMINIKAŃSKA time: 46.333333333333336 minutes
For:  GALERIA DOMINIKAŃSKA to: DWORZEC AUTOBUSOWY time: 53.333333333333336 minutes
For:  ['DWORZEC AUTOBUSOWY' 'FAT' 'GALERIA DOMINIKAŃSKA'] time: 53.333333333333336 minutes
For:  DWORZEC AUTOBUSOWY to: GALERIA DOMINIKAŃSKA time: 6.333333333333333 minutes
For:  GALERIA DOMINIKAŃSKA to: FAT time: 67.33333333333333 minutes
For:  FAT to: DWORZEC AUTOBUSOWY time: 82.33333333333333 minutes
For:  ['DWORZEC AUTOBUSOWY' 'GALERIA DOMINIKAŃSKA' 'FAT'] time: 82.33333333333333 minutes
For:  FAT to: DWORZEC AUTOBUSOWY time: 16.333333333333332 minutes
For:  DWORZEC AUTOBUSOWY to: GALERIA DOMINIKAŃSKA time: 24.333333333333332 minutes
For:  GALERIA DOMINIKAŃSKA to: FAT time: 47.333333333333336 minutes
For:  ['FAT' 'DWORZEC AUTOBUSOWY' 'GALERIA DOMINIKAŃSKA'] time: 47.333333333333336 minutes
For:  FAT to: GALERIA DOMINIKAŃSKA time: 20.333333333333332 minutes
For:  GALERI